In [2]:
# ============================================
# Interactive BED File Analyzer (HTML Version)
# ============================================

from IPython.display import clear_output, display, HTML
import time
import os

# Ask user for file path
file_path = input('Please enter the BED file path: ').strip()

# Read the BED file and collect information
data = []
try:
    with open(file_path, 'r') as myfile:
        for line in myfile:
            if line.strip() and not line.startswith('#'):
                parts = line.strip().split()
                if len(parts) >= 4:
                    chrom = parts[0]
                    quality = float(parts[3])
                    data.append((chrom, quality))
except FileNotFoundError:
    display(HTML("<h3 style='color:red;'>❌ File not found! Please check the path and try again.</h3>"))
    exit()

# Ensure output folder exists
os.makedirs("../Data", exist_ok=True)

# Main menu loop
while True:
    print("""
          Select an option:
        [R] Display read counts in each chromosome
        [D] Display average read quality for each chromosome
        [X] Exit
      """)

    choice = input("Enter choice (R/D/X): ").strip().lower()

    # --- OPTION R: Read Counts ---
    if choice == 'r':
        clear_output(wait=True)
        display(HTML("<h3 style='color:blue;'>🔄 Calculating read counts per chromosome...</h3>"))
        time.sleep(1)

        counts_dict = {}
        for chrom, _ in data:
            counts_dict[chrom] = counts_dict.get(chrom, 0) + 1

        # Sort by count (descending)
        sorted_counts = dict(sorted(counts_dict.items(), key=lambda x: x[1], reverse=True))

        # Build HTML table
        table_html = "<table border='1' style='border-collapse:collapse;width:60%;text-align:left;'>"
        table_html += "<tr style='background-color:#a5d6a7;'><th>Chromosome</th><th>Read Count</th></tr>"
        for chrom, count in sorted_counts.items():
            table_html += f"<tr><td>{chrom}</td><td>{count}</td></tr>"
        table_html += "</table>"

        # Save output file
        output_file = '../Data/read_counts.txt'
        with open(output_file, 'w') as outfile:
            outfile.write("Chromosome\tRead_count\n")
            for chrom, count in sorted_counts.items():
                outfile.write(f"{chrom}\t{count}\n")

        clear_output(wait=True)
        display(HTML(f"""
        <div style='border:2px solid #4CAF50;border-radius:10px;padding:10px;background-color:#f0fdf4;'>
            <h3 style='color:#2E7D32;'>📊 Read Counts per Chromosome</h3>
            {table_html}
            <p style='color:gray;'>✅ Results saved to: <b>{output_file}</b></p>
        </div>
        """))

    # --- OPTION D: Average Read Quality ---
    elif choice == 'd':
        clear_output(wait=True)
        display(HTML("<h3 style='color:blue;'>🔄 Calculating average read quality...</h3>"))
        time.sleep(1)

        total_quality = {}
        read_count = {}

        for chrom, quality in data:
            total_quality[chrom] = total_quality.get(chrom, 0) + quality
            read_count[chrom] = read_count.get(chrom, 0) + 1

        # Build HTML table
        table_html = "<table border='1' style='border-collapse:collapse;width:60%;text-align:left;'>"
        table_html += "<tr style='background-color:#90caf9;'><th>Chromosome</th><th>Average Quality</th></tr>"
        for chrom in total_quality:
            avg = total_quality[chrom] / read_count[chrom]
            table_html += f"<tr><td>{chrom}</td><td>{avg:.3f}</td></tr>"
        table_html += "</table>"

        # Save output file
        output_file = '../Data/quality_average.txt'
        with open(output_file, 'w') as outf:
            outf.write("Chromosome\tAverage_Quality\n")
            for chrom in total_quality:
                avg = total_quality[chrom] / read_count[chrom]
                outf.write(f"{chrom}\t{avg:.3f}\n")

        clear_output(wait=True)
        display(HTML(f"""
        <div style='border:2px solid #2196F3;border-radius:10px;padding:10px;background-color:#E3F2FD;'>
            <h3 style='color:#1565C0;'>🎯 Average Read Quality per Chromosome</h3>
            {table_html}
            <p style='color:gray;'>✅ Results saved to: <b>{output_file}</b></p>
        </div>
        """))

    # --- OPTION X: Exit ---
    elif choice == 'x':
        clear_output(wait=True)
        display(HTML("<h3 style='color:green;'>👋 Exiting program. Goodbye!</h3>"))
        break

    # --- Invalid Option ---
    else:
        clear_output(wait=True)
        display(HTML("<h3 style='color:red;'>⚠️ Invalid choice. Please enter R, D, or X.</h3>"))
